In [1]:
import pandas as pd
from persiantools import characters
from glob import glob

In [2]:
path_raw = r'C:\Users\Sarv\Desktop\vs_code\Git\ETL-pipeline\data\external\security_info.csv'

In [111]:
def clean_cols(df):
    dropped_cols = ['code_stock_5','code_company_4','stock_name_2','trade_purpose',
                    'CEO','address','phone_number','FAX','offic_address','shareholder_affairs_address',
                    'website','email','auditor','capital','financial_manager']

    new_cols=['stock_code','company_name_en','company_name_fa','namad','stock_name',
              'company_code','market','code_tablo','sector_code','sector_name','subsector_code',
              'subsector_name','stock_tsetmc_code','financial_year','national_id']
    df.drop(columns=dropped_cols,inplace=True)
    df.market=df.market.replace("-",None)
    df.year=df.year.replace("--/--",None)
    df.shenaseh_meli=df.shenaseh_meli.astype(str).apply(lambda x: x.split('.')[0])
    df.shenaseh_meli.replace('nan',None, inplace=True)
    df.columns=new_cols
    df.insert(0,'security_id',df['national_id'].fillna(df['stock_tsetmc_code']))
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True,drop=True)
    return df

In [4]:
def ar_to_fa(df):
    fa_cols = ['company_name_fa','namad','stock_name',
            'sector_name','subsector_name']
    df[fa_cols]=df[fa_cols].applymap(lambda x: characters.ar_to_fa(x))
    return df

In [47]:
def security_type(df):
    Tese = df[(df['code_tablo']==4)&((df['stock_name'].str.contains('تسه'))|(df['stock_name'].str.contains('تمل')))]
    Sandogh = df[(df['code_tablo']==4)&((df['company_name_fa'].str.contains('صندوق'))|(df['company_name_fa'].str.contains('خصو'))|(df['company_name_fa'].str.contains('ص سرمايه گذا'))|(df['company_name_fa'].str.contains('ص.س. زمين')))]
    Akhza = df[(df['code_tablo']==4)&(df['company_name_fa'].str.contains('اسناد'))]
    Govahi_Kala = df[(df['code_tablo']==4)&(df['market'].str.contains('کالا'))&(df['company_name_fa'].str.contains('صندوق')==False)]
    Arad = df[(df['code_tablo']==4)&(df['namad'].str.contains('اراد'))]
    Afad = df[(df['code_tablo']==4)&(df['namad'].str.contains('افاد'))]
    Jasourane = df[(df['code_tablo']==4)&((df['company_name_fa'].str.contains('جسورانه'))|(df['company_name_fa'].str.contains('ص.س.ج')))]
    Sukuk = df[df['code_tablo']==9]
    Option = df[df['code_tablo']==8]
    Hagh_Taghadom=df[(df['namad'].str.endswith('ح'))|(df['namad'].str.startswith('ح.'))|(df['namad'].str.startswith('ح .'))]
    Stock = df[~df.isin(pd.concat([Tese,Sandogh,Akhza,Govahi_Kala,Arad,Afad,Jasourane,Sukuk,Option,Hagh_Taghadom]))].dropna(how='all')

    df.insert(5,'security_type',None)
    df['security_type'].loc[Tese.index]='اوراق تسهیلات مسکن'
    df['security_type'].loc[Sandogh.index]='صندوق سرمایه گذاری'
    df['security_type'].loc[Akhza.index]='اسناد خزانه اسلامی'
    df['security_type'].loc[Govahi_Kala.index]='گواهی سپرده کالایی'
    df['security_type'].loc[Arad.index]='اوراق مرابحه عام دولت'
    df['security_type'].loc[Afad.index]='اوراق منفعت دولت'
    df['security_type'].loc[Jasourane.index]='صندوق جسورانه'
    df['security_type'].loc[Sukuk.index]='صکوک شرکتی و گام'
    df['security_type'].loc[Option.index]='اختیار معامله'
    df['security_type'].loc[Hagh_Taghadom.index]='حق تقدم'
    df['security_type'].loc[Stock.index]='سهام'
    
    return df


In [136]:
df=pd.read_csv(path_raw)
df.pipe(clean_cols).pipe(ar_to_fa).pipe(security_type)
df.iloc[809,0]='10101924696-2'
df.iloc[802,0]='10101634157-2'
df.iloc[475,0]='10102989222'
df.to_csv(r'C:\Users\Sarv\Desktop\vs_code\Git\ETL-pipeline\data\interim\security_info_mod.csv',index=False)

C:\Users\Sarv\AppData\Local\Temp\ipykernel_16976\3052870207.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['security_type'].loc[Tese.index]='اوراق تسهیلات مسکن'
C:\Users\Sarv\AppData\Local\Temp\ipykernel_16976\3052870207.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['security_type'].loc[Sandogh.index]='صندوق سرمایه گذاری'
C:\Users\Sarv\AppData\Local\Temp\ipykernel_16976\3052870207.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view